In [1]:
import sys, os

sys.path.append(os.path.abspath("..")) # adding experiment folder to system path (easy import)
sys.path.append(os.path.abspath("../../Spotifier Server")) # adding detection server folder to system path (easy import)
from Helper.experiment_helper import * # helper methods
from Flow.backbone_detector import BackboneDetector

# Parameters

In [2]:
SPOTIFIER_ROOT = os.path.abspath("../../")                #Root directory of the spotifier
PDS_DATASET_DIR = os.path.abspath("../../../Dataset/PDS") #Root directory that contains all the data and meta data files

# Path to the pickle files containing paths of all input images segmented into folds
PK_LOT_VAL_FOLD_DICT_PATH = os.path.join(PDS_DATASET_DIR,"Derived Dataset", "K Folded Pickles","Filtered_validation_100_fold.pickle")
CNR_OCC_100_FOLD_DICT_PATH = os.path.join(PDS_DATASET_DIR,"Derived Dataset", "K Folded Pickles","Occlusion_100_fold_cnr.pickle")

# Path to the location of the image datasets
CNR_DATASET_DIR = os.path.join(PDS_DATASET_DIR, "Amato", "CNR EXT")
PK_LOT_CSV_PATH = os.path.join(PDS_DATASET_DIR, 'Derived Dataset', 'Pklot_dataset.csv')

# Experiment Variables

In [3]:
# Exectution variables

model_name =  "m2"  # Keyword for model name, change this string to use different models (mAlex, vgg-f, m2)
Run_name =    "test"  # Name of experiment session

result_dir =  Run_name # Directory wher all the execution variable will be stored
fold_range =  (0,10)  # Range of folds/segments of datasets to be used in the experiment, start and end (+1) index

In [4]:
# Read the experiment data (image path) in the form of dictionary where each key represent one type of whether [Cloudy, Rainy & Sunny] 

input_image_paths = read_k_fold_dataset(CNR_OCC_100_FOLD_DICT_PATH) # Read all image paths segmeneted into folds for CNR image dataset
dataset_location_path = CNR_DATASET_DIR                             # Location of the all image dateset for CNR

In [5]:
# Detection Model

root_dir = os.path.join(SPOTIFIER_ROOT, "Spotifier Server")   # Directory where model is located
backbone_obj = BackboneDetector(model_name, root_dir)         # Instantiate the object of the detector

Loading model
Model Loaded


# Experiment Execution

In [6]:
occ_init_point = 0.43         # Starting fraction/percentage of occlusion (it will be fraction of height and width of the original image)
occ_increment = 0.01          # Increment in initial occlusion point
occlusion_result_dict = {}    # Dictionary to save session meta data

for occ_ind in range(50):     # For loop over entire occlusion dataset with fixed size of occlusion box, in next iteration increase the size and repate the process for 50 iterations 
    positive_list = []        # No of correct detections (meta data)
    total_list = []           # Total no of correct detections (meta data)
    mean_confidence_list = [] # Mean confidence (meta data)
    
    occlusion_fraction = occ_init_point + occ_ind*occ_increment               # Occlusion fraction determine size of the occlusion box, it is proportional to the image dimenions
    occ_ind_dir = result_dir+"//"+str(occ_ind+1)+"_"+str(occlusion_fraction)  # Path to the detection output of each occlusion size run that is stored in exclusive directory  
    file_name = result_dir+str(round(occlusion_fraction,3))                   
    if not os.path.isdir(occ_ind_dir):                                      
        os.makedirs(occ_ind_dir)
    Run_name = occ_ind_dir+"//"+file_name
    for fold_ind in range(fold_range[0], fold_range[1]):
        for weather in input_image_paths.keys():
            img_path_list = input_image_paths[weather][fold_ind][0:2]        # Extract input data and targets from the dictionary
            name = weather+"_"+str(fold_ind+1)                               # Name to be printed while monitring execution                                                                                                                         
            
        # Main Detection
            conf_matrix, report, accuracy, y_list, conf_list, mean_backbone_time, total_detect_time = get_validation_conf_matrix_from_fold(img_path_list, backbone_obj, occlusion_fract = occlusion_fraction, dataset_img_path = dataset_location_path ,banner = name)
        # Saving meta data
            positive_list.append(conf_matrix[0,0] + conf_matrix[1,1])
            total_list.append(np.sum(conf_matrix))
            mean_confidence_list.append(np.mean(conf_list))
        # Saving the detection results    
            write_response_in_file(Run_name, conf_matrix, report, accuracy, y_list, conf_list, mean_backbone_time, total_detect_time, name)
    occlusion_result_dict[occlusion_fraction] = [np.sum(positive_list)/np.sum(total_list), np.mean(mean_confidence_list)]